## Loading essential libraries

In [1]:
import pandas as pd
import seaborn as sns
import requests
import matplotlib.pyplot as plt
import xlsxwriter
# pd.set_option('display.max_rows',  35)
# pd.set_option('display.max_columns' , 64 )

## Download data from website - ICO (International Coffee Organization)

In [2]:
#extract data from website for given url 0- https://www.ico.org/new_historical.asp

df_list = []
url_list = ['https://www.ico.org/historical/1990%20onwards/Excel/1a%20-%20Total%20production.xlsx',
            'https://www.ico.org/historical/1990%20onwards/Excel/1b%20-%20Domestic%20consumption.xlsx',
            'https://www.ico.org/historical/1990%20onwards/Excel/2a%20-%20Exports%20-%20calendar%20year.xlsx',
            'https://www.ico.org/historical/1990%20onwards/Excel/2b%20-%20Imports.xlsx',
            'https://www.ico.org/historical/1990%20onwards/Excel/4b%20-%20Disappearance.xlsx'
           ]
def get_data_from_url(url_list):

    """
    Extract data from website ICO.org
    Save each dataset as pandas data frame on df_list for easy access
    
    """
    for url in url_list:
        response = requests.get(url) 
        response.raise_for_status() # check for any error 
        df = pd.read_excel(url, header = 3)
        df_list.append(df)
        
get_data_from_url(url_list)

df_total_production = df_list[0] # total production by country
df_domestic_consumption = df_list[1] # consuption by export
df_export = df_list[2]
df_import = df_list[3]
df_consumption_in_import = df_list[4]


C:\Users\JASNYP\Desktop\Python Projects\GlobalCoffeeTrade\venv\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\JASNYP\Desktop\Python Projects\GlobalCoffeeTrade\venv\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## Data preprocessing and cleaning for 'total production'

In [3]:
# df_total_production

In [4]:
#I make the cleaning part realy short after I looked deep into this dataset

df_total_production = df_total_production.rename(columns={'Crop year' : 'Country', 'Unnamed: 1' : 'Coffee_type'})
df_total_production['Coffee_type'].replace({'(R/A)' : 'Arabica/Robusta', '(A)' : 'Arabica', '(R)' : 'Robusta' , '(A/R)' : 'Arabica/Robusta', 0 : 'Arabica' }, inplace = True )
df_total_production.drop(index=[0, 1, 14, 16, 17, 22, 25,  26, 57, 61, 62 ,63] , inplace = True ) 
df_total_production.reset_index(drop = True, inplace = True )

In [5]:
#  In order to obtain kg values, data have to be multiplaying by 60 kg coffee bags and 1 000 value ( production is given as thousand 60kg bags )
years = df_total_production.iloc[:, 2:]
years = years.apply(lambda x: round(x, 0))
years = years.apply(lambda x: x*60*1000)

country_coffee = df_total_production.iloc[:, 0:2]
df_total_production = pd.concat([country_coffee , years], axis = 1 ).reset_index(drop = True )

In [6]:
df_total_production['Total_production'] =  df_total_production.iloc[:, 2:].sum(axis=1)
df_total_production.head()

,Country,Coffee_type,1990/91,1991/92,1992/93,1993/94,1994/95,1995/96,1996/97,1997/98,...,2011/12,2012/13,2013/14,2014/15,2015/16,2016/17,2017/18,2018/19,2019/20,Total_production
0,Angola,Arabica/Robusta,3.000000e+06,4.740000e+06,4.680000e+06,1.980000e+06,4.620000e+06,3.720000e+06,4.260000e+06,3.840000e+06,...,1.740000e+06,1.980000e+06,2.100000e+06,2.340000e+06,2.460000e+06,2.700000e+06,2.100000e+06,2.520000e+06,3.120000e+06,8.208000e+07
1,Bolivia (Plurinational State of),Arabica,7.380000e+06,6.240000e+06,7.200000e+06,3.060000e+06,7.020000e+06,8.520000e+06,7.500000e+06,8.460000e+06,...,7.920000e+06,6.300000e+06,7.200000e+06,6.000000e+06,5.040000e+06,4.680000e+06,5.040000e+06,4.980000e+06,4.860000e+06,2.070000e+08
2,Brazil,Arabica/Robusta,1.637160e+09,1.637580e+09,2.076180e+09,1.690020e+09,1.691520e+09,1.083600e+09,1.751820e+09,1.568880e+09,...,2.915520e+09,3.325080e+09,3.281340e+09,3.198300e+09,3.172260e+09,3.407280e+09,3.164400e+09,3.907860e+09,3.492660e+09,7.508298e+10
3,Burundi,Arabica/Robusta,2.922000e+07,4.002000e+07,3.720000e+07,2.358000e+07,3.984000e+07,2.604000e+07,2.406000e+07,1.500000e+07,...,1.224000e+07,2.436000e+07,9.780000e+06,1.488000e+07,1.614000e+07,1.176000e+07,1.212000e+07,1.224000e+07,1.632000e+07,6.236400e+08
4,Ecuador,Arabica/Robusta,9.024000e+07,1.274400e+08,7.110000e+07,1.241400e+08,1.425600e+08,1.132800e+08,1.195800e+08,7.146000e+07,...,4.950000e+07,4.968000e+07,3.996000e+07,3.864000e+07,3.864000e+07,3.870000e+07,3.744000e+07,2.976000e+07,3.354000e+07,1.900380e+09


In [7]:
df_total_production

,Country,Coffee_type,1990/91,1991/92,1992/93,1993/94,1994/95,1995/96,1996/97,1997/98,...,2011/12,2012/13,2013/14,2014/15,2015/16,2016/17,2017/18,2018/19,2019/20,Total_production
0,Angola,Arabica/Robusta,3.000000e+06,4.740000e+06,4.680000e+06,1.980000e+06,4.620000e+06,3.720000e+06,4.260000e+06,3.840000e+06,...,1.740000e+06,1.980000e+06,2.100000e+06,2.340000e+06,2.460000e+06,2.700000e+06,2.100000e+06,2.520000e+06,3.120000e+06,8.208000e+07
1,Bolivia (Plurinational State of),Arabica,7.380000e+06,6.240000e+06,7.200000e+06,3.060000e+06,7.020000e+06,8.520000e+06,7.500000e+06,8.460000e+06,...,7.920000e+06,6.300000e+06,7.200000e+06,6.000000e+06,5.040000e+06,4.680000e+06,5.040000e+06,4.980000e+06,4.860000e+06,2.070000e+08
2,Brazil,Arabica/Robusta,1.637160e+09,1.637580e+09,2.076180e+09,1.690020e+09,1.691520e+09,1.083600e+09,1.751820e+09,1.568880e+09,...,2.915520e+09,3.325080e+09,3.281340e+09,3.198300e+09,3.172260e+09,3.407280e+09,3.164400e+09,3.907860e+09,3.492660e+09,7.508298e+10
3,Burundi,Arabica/Robusta,2.922000e+07,4.002000e+07,3.720000e+07,2.358000e+07,3.984000e+07,2.604000e+07,2.406000e+07,1.500000e+07,...,1.224000e+07,2.436000e+07,9.780000e+06,1.488000e+07,1.614000e+07,1.176000e+07,1.212000e+07,1.224000e+07,1.632000e+07,6.236400e+08
4,Ecuador,Arabica/Robusta,9.024000e+07,1.274400e+08,7.110000e+07,1.241400e+08,1.425600e+08,1.132800e+08,1.195800e+08,7.146000e+07,...,4.950000e+07,4.968000e+07,3.996000e+07,3.864000e+07,3.864000e+07,3.870000e+07,3.744000e+07,2.976000e+07,3.354000e+07,1.900380e+09
5,Indonesia,Arabica/Robusta,4.464600e+08,5.095800e+08,3.341400e+08,4.045800e+08,3.220800e+08,2.743800e+08,4.932600e+08,4.572600e+08,...,4.133400e+08,7.842000e+08,7.740600e+08,6.567600e+08,7.551000e+08,6.924600e+08,6.511200e+08,5.770800e+08,6.859800e+08,1.540488e+10
6,Madagascar,Robusta,5.892000e+07,5.598000e+07,6.732000e+07,2.652000e+07,3.846000e+07,4.710000e+07,5.094000e+07,3.744000e+07,...,3.510000e+07,3.000000e+07,3.504000e+07,3.006000e+07,2.484000e+07,2.718000e+07,2.424000e+07,2.274000e+07,2.298000e+07,1.045560e+09
7,Malawi,Arabica,6.300000e+06,7.440000e+06,8.220000e+06,3.720000e+06,5.040000e+06,5.460000e+06,2.940000e+06,3.660000e+06,...,1.560000e+06,1.380000e+06,1.680000e+06,1.500000e+06,1.260000e+06,1.200000e+06,8.400000e+05,7.800000e+05,9.600000e+05,8.226000e+07
8,Papua New Guinea,Arabica/Robusta,5.778000e+07,4.482000e+07,5.400000e+07,6.114000e+07,6.834000e+07,6.012000e+07,6.534000e+07,6.444000e+07,...,8.484000e+07,4.296000e+07,5.010000e+07,4.788000e+07,4.272000e+07,7.026000e+07,4.404000e+07,5.580000e+07,4.512000e+07,1.803120e+09
9,Paraguay,Arabica,7.860000e+06,4.800000e+06,3.240000e+06,4.020000e+06,1.500000e+06,1.560000e+06,1.860000e+06,2.940000e+06,...,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06,6.222000e+07


## Data preprocessing and cleaning for 'domestic consumption'

In [8]:
# df_domestic_consumption.head()

In [9]:
df_domestic_consumption.rename(columns={'Crop year' : 'Country', 'Unnamed: 1' : 'Coffee_type'}, inplace = True )
df_domestic_consumption['Coffee_type'].replace({'(R/A)' : 'Arabica/Robusta', '(A)' : 'Arabica', '(R)' : 'Robusta' , '(A/R)' : 'Arabica/Robusta', 0 : 'Arabica' }, inplace = True )
df_domestic_consumption.drop(index=[0, 1, 14, 16, 22, 25, 57, 17, 26, 61, 62 ,63] , inplace = True ) 
df_domestic_consumption.reset_index(drop = True, inplace = True )
# df_domestic_consumption

In [10]:
years = df_domestic_consumption.iloc[:, 2:]
countries = df_domestic_consumption.iloc[:, :1]
years = years.apply(lambda x: x*60*1000) 
years = years.apply(lambda x: round(x, 0))

In [11]:
df_domestic_consumption = pd.concat([countries , years], axis = 1 ).reset_index(drop = True ) 
df_domestic_consumption.tail(10)

,Country,1990/91,1991/92,1992/93,1993/94,1994/95,1995/96,1996/97,1997/98,1998/99,...,2010/11,2011/12,2012/13,2013/14,2014/15,2015/16,2016/17,2017/18,2018/19,2019/20
42,Nigeria,1650000.0,2145000.0,2145000.0,2400000.0,2400000.0,2400000.0,2400000.0,2400000.0,2400000.0,...,2400000.0,2400000.0,2400000.0,2400000.0,2400000.0,2400000.0,2400000.0,2400000.0,2400000.0,2400000.0
43,Panama,3780000.0,4020000.0,4020000.0,4020000.0,4600020.0,4600020.0,4600020.0,4836900.0,4020000.0,...,4020000.0,4020000.0,4020000.0,4020000.0,4020000.0,4020000.0,4020000.0,4020000.0,4020000.0,4020000.0
44,Sierra Leone,540000.0,540000.0,540000.0,300000.0,300000.0,480000.0,480000.0,300000.0,300000.0,...,300000.0,300000.0,300000.0,300000.0,300000.0,300000.0,300000.0,300000.0,300000.0,300000.0
45,Sri Lanka,4000020.0,4020000.0,1680000.0,1200000.0,1200000.0,1200000.0,1200000.0,1200000.0,1800000.0,...,2100000.0,2100000.0,2100000.0,2100000.0,2100000.0,2100000.0,2100000.0,2100000.0,2100000.0,2100000.0
46,Thailand,11280000.0,13200000.0,13980000.0,15000000.0,16980000.0,25020000.0,19980000.0,23400000.0,25980000.0,...,46500000.0,66000000.0,67800000.0,72000000.0,75000000.0,78000000.0,81000000.0,82500000.0,84000000.0,84000000.0
47,Togo,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,120000.0,...,120000.0,12600.0,14520.0,14280.0,11700.0,13200.0,15240.0,15240.0,15240.0,15600.0
48,Trinidad & Tobago,480000.0,690000.0,600000.0,600000.0,840000.0,840000.0,840000.0,840000.0,840000.0,...,600000.0,600000.0,600000.0,600000.0,600000.0,600000.0,600000.0,600000.0,600000.0,600000.0
49,Venezuela,46943400.0,48928800.0,50998200.0,53155800.0,55404000.0,57747600.0,60190800.0,62736600.0,65390400.0,...,99000000.0,99000000.0,99000000.0,99000000.0,99000000.0,99000000.0,99000000.0,96000000.0,93000000.0,76500000.0
50,Viet Nam,9000000.0,13800000.0,15000000.0,16020000.0,16020000.0,16910700.0,17850960.0,18843480.0,19891140.0,...,94980000.0,99000000.0,109500000.0,120000000.0,132000000.0,138000000.0,144000000.0,150000000.0,156000000.0,159000000.0
51,Yemen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7800000.0,7800000.0,7800000.0,7800000.0,7500000.0,6900000.0,6000000.0,5400000.0,3900000.0,3840000.0


In [12]:
df_domestic_consumption['Total_consumption'] = df_domestic_consumption.iloc[: , 2: ].sum(axis = 1)
df_domestic_consumption.head()

,Country,1990/91,1991/92,1992/93,1993/94,1994/95,1995/96,1996/97,1997/98,1998/99,...,2011/12,2012/13,2013/14,2014/15,2015/16,2016/17,2017/18,2018/19,2019/20,Total_consumption
0,Angola,1200000.0,1800000.0,2100000.0,1200000.0,1500000.0,600000.0,1200000.0,2400000.0,1800000.0,...,1.800000e+06,1.800000e+06,1.800000e+06,1.800000e+06,1.800000e+06,1.800000e+06,1.800000e+06,1.800000e+06,1.800000e+06,4.530000e+07
1,Bolivia (Plurinational State of),1500000.0,1620000.0,1650000.0,1710000.0,1770000.0,1830000.0,1890000.0,1950000.0,1980000.0,...,2.940000e+06,3.030000e+06,3.120000e+06,3.210000e+06,3.300000e+06,3.420000e+06,3.510000e+06,3.600000e+06,3.660000e+06,7.368000e+07
2,Brazil,492000000.0,510000000.0,534000000.0,546000000.0,558000000.0,606000000.0,660000000.0,690000000.0,732000000.0,...,1.183200e+09,1.219800e+09,1.205100e+09,1.219980e+09,1.230480e+09,1.273500e+09,1.319820e+09,1.332000e+09,1.320000e+09,2.733270e+10
3,Burundi,120000.0,96000.0,102000.0,114600.0,120000.0,120000.0,120000.0,120000.0,120000.0,...,1.200000e+05,1.200000e+05,1.200000e+05,1.200000e+05,1.200000e+05,1.200000e+05,1.200000e+05,1.200000e+05,1.200000e+05,3.292020e+06
4,Ecuador,21000000.0,21000000.0,21000000.0,21000000.0,21000000.0,21000000.0,18000000.0,18000000.0,18000000.0,...,9.000000e+06,9.000000e+06,9.300000e+06,9.300000e+06,9.300000e+06,9.300000e+06,9.300000e+06,9.300000e+06,8.940000e+06,3.605400e+08


## Cleaning and preprocessing data for 'export'

In [13]:
# df_export.tail()

In [14]:
df_export.rename(columns ={'Calendar years' : 'Country'}, inplace = True )
df_export.drop(index = [55, 56,57], inplace = True)
df_export.iloc[: , 2:] = df_export.iloc[: , 2:].apply(lambda x: x*60*1000)
df_export.iloc[: , 2:] = df_export.iloc[:,2:].apply(lambda x: round(x,0))
df_export['Total_export'] = df_export.iloc[:,2:].sum(axis = 1)

In [15]:
df_export.tail(10)

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total_export
45,Thailand,1001.435000,32456220.0,69826560.0,58906920.0,68162760.0,72571380.0,52361160.0,63973800.0,46681860.0,...,1.456524e+07,2.098316e+07,2.958462e+06,1.642228e+06,3.469974e+07,7.857154e+06,8.467580e+06,1.548802e+07,1.045792e+07,9.105868e+08
46,Timor-Leste,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.323270e+06,3.352623e+06,4.214395e+06,6.663742e+06,4.061160e+06,4.071198e+06,5.176673e+06,7.720108e+06,7.031863e+06,6.807671e+07
47,Togo,238.838000,9290040.0,18790620.0,13097040.0,9947700.0,12018960.0,5129400.0,18788220.0,9948060.0,...,9.305050e+06,8.843427e+06,5.445392e+06,8.038118e+06,1.068629e+07,4.492610e+06,1.639726e+06,6.662661e+06,3.509607e+06,2.770949e+08
48,Trinidad & Tobago,30.232000,460320.0,147060.0,584640.0,324000.0,243600.0,225420.0,237120.0,313140.0,...,1.799000e+04,8.689400e+04,1.491550e+05,1.282280e+05,1.159360e+05,1.058730e+05,1.080000e+05,1.722340e+05,1.482650e+05,4.723250e+06
49,Uganda,2352.680000,125705580.0,117662040.0,109744860.0,202125900.0,184755660.0,279270180.0,210116520.0,197160720.0,...,1.885207e+08,1.611142e+08,2.203127e+08,2.065411e+08,2.157364e+08,2.125858e+08,2.864420e+08,2.533909e+08,2.715686e+08,5.490981e+09
50,Venezuela,277.795000,7673340.0,8149080.0,31500120.0,23159760.0,5805120.0,29674020.0,6272220.0,16053780.0,...,2.802500e+04,1.388870e+05,3.458080e+05,4.925000e+03,3.021800e+04,0.000000e+00,0.000000e+00,4.306380e+06,3.194004e+06,2.245440e+08
51,Viet Nam,1145.234001,72057120.0,116256660.0,124310220.0,163254660.0,212784300.0,226764900.0,370670040.0,388002720.0,...,1.063044e+09,1.375180e+09,1.183066e+09,1.565829e+09,1.316610e+09,1.783277e+09,1.505530e+09,1.883104e+09,1.644010e+09,2.485583e+10
52,Yemen,50.000000,4300020.0,2219340.0,3051180.0,5570400.0,4612440.0,2520420.0,4787340.0,4247460.0,...,2.116601e+06,3.759096e+06,3.320948e+06,3.419343e+06,3.133945e+06,2.274076e+06,2.785326e+06,2.527088e+06,7.308310e+05,9.816125e+07
53,Zambia,24.360000,1751880.0,1830840.0,1384320.0,1437240.0,1244520.0,2149440.0,2333400.0,1998480.0,...,5.401800e+05,4.245440e+05,6.152930e+05,3.146510e+05,1.763730e+05,5.874670e+05,7.541390e+05,1.570139e+06,1.636607e+06,7.474293e+07
54,Zimbabwe,250.772000,14359440.0,9739080.0,4189680.0,1904640.0,9418500.0,9021720.0,6540480.0,9196080.0,...,2.737320e+05,2.402120e+05,1.308480e+05,6.781320e+05,6.641410e+05,7.372250e+05,1.409520e+05,2.883190e+05,3.760990e+05,1.212462e+08


## Cleaning and preprocessing for 'import'

In [16]:
# df_import

In [48]:
df_import.rename(columns = {'Calendar years' : 'Country'}, inplace = True)
df_import.drop(index = [0, 30, 38,39,40], inplace = True)
df_import.reset_index(drop = True, inplace = True)
df_import.iloc[:, 2:] = df_import.iloc[:, 2:].apply(lambda x: x*60*1000)
df_import.iloc[:, 2:] = df_import.iloc[:, 2:].apply(lambda x: round(x,0))
df_import['Total_import'] = df_import.iloc[:,2:].sum(axis = 1)
df_import.Country = df_import.Country.str.strip()

In [49]:
df_import.tail()

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total_import
31,Russian Federation,0.000000,0.000000e+00,8.291148e+07,1.076968e+08,103543801.0,1.036886e+08,6.767832e+07,1.311173e+08,1.039151e+08,...,2.530610e+08,2.504847e+08,2.646116e+08,2.848170e+08,2.826023e+08,3.139729e+08,3.280670e+08,3.172525e+08,3.549980e+08,5.731071e+09
32,Switzerland,1170.508002,6.781752e+07,6.607506e+07,6.359268e+07,65964360.0,6.100974e+07,6.669762e+07,5.824632e+07,6.671010e+07,...,1.498748e+08,1.486774e+08,1.600050e+08,1.585864e+08,1.648715e+08,1.689616e+08,1.742171e+08,1.851611e+08,1.937030e+08,3.142347e+09
33,Tunisia,100.999998,5.412060e+06,6.634980e+06,6.037440e+06,8236020.0,8.206980e+06,8.619000e+06,9.877020e+06,8.845020e+06,...,2.572692e+07,2.633542e+07,2.633804e+07,2.760167e+07,2.752242e+07,2.793318e+07,3.276935e+07,3.197799e+07,3.049992e+07,4.864041e+08
34,United Kingdom,2898.298008,1.683548e+08,1.852903e+08,1.918189e+08,207951601.0,1.684034e+08,1.742072e+08,1.757603e+08,1.885445e+08,...,2.509938e+08,2.475632e+08,2.523655e+08,2.591217e+08,2.937244e+08,3.031376e+08,2.993423e+08,3.412716e+08,3.332618e+08,6.557718e+09
35,United States of America,21006.786010,1.190378e+09,1.376342e+09,1.159723e+09,970274701.0,1.026411e+09,1.166672e+09,1.220565e+09,1.261824e+09,...,1.565604e+09,1.563370e+09,1.620932e+09,1.653930e+09,1.662460e+09,1.730270e+09,1.769273e+09,1.735092e+09,1.851244e+09,4.124721e+10


## Cleaning and preprocessing for 'consumption in importing countries'

In [19]:
# df_consumption_in_import

In [45]:
df_consumption_in_import.rename(columns = {'Calendar years' : 'Country'}, inplace = True)
df_consumption_in_import.drop(index = [0, 30, 38,39,40], inplace = True)
df_consumption_in_import.reset_index(drop = True, inplace = True)
df_consumption_in_import.iloc[:, 2:] = df_consumption_in_import.iloc[:, 2:].apply(lambda x: x*60*1000)
df_consumption_in_import.iloc[:, 2:] = df_consumption_in_import.iloc[:, 2:].apply(lambda x: round(x,0))
df_consumption_in_import['Total_consumption'] = df_consumption_in_import.iloc[:,2:].sum(axis = 1)
df_consumption_in_import.Country = df_consumption_in_import.Country.str.strip()

In [46]:
df_consumption_in_import.head()

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total_consumption
0,European Union,31881.539292,2.119303e+09,2.143412e+09,2.265518e+09,2.052593e+09,2.192105e+09,2.258100e+09,2.214659e+09,2.242578e+09,...,2.269859e+09,2.283168e+09,2.325453e+09,2.383029e+09,2.292444e+09,2.443253e+09,2.350136e+09,2.480530e+09,2.475721e+09,6.579112e+10
1,Austria,1340.123985,7.812516e+07,7.274004e+07,8.010006e+07,6.517314e+07,6.070620e+07,6.454728e+07,6.522478e+07,6.628429e+07,...,6.704074e+07,7.614117e+07,7.494337e+07,6.808258e+07,6.815031e+07,7.159482e+07,6.969710e+07,7.031960e+07,7.040110e+07,1.824067e+09
2,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.606812e+07,5.487819e+07,7.470989e+07,7.553621e+07,6.441664e+07,9.012749e+07,7.800329e+07,8.189560e+07,7.112258e+07,1.460688e+09
3,Belgium/Luxembourg,1124.109984,4.248942e+07,4.178238e+07,4.736622e+07,5.749290e+07,6.438090e+07,6.231822e+07,5.317583e+07,7.856439e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.475703e+08
4,Bulgaria,101.782998,1.005660e+06,1.049268e+07,1.961730e+07,2.116584e+07,2.360448e+07,1.478850e+07,1.649508e+07,1.944960e+07,...,2.157118e+07,2.255648e+07,2.511641e+07,2.257418e+07,2.437543e+07,2.714076e+07,2.490071e+07,2.554317e+07,2.538400e+07,6.190538e+08


In [47]:
df_consumption_in_import.Country.unique()

array(['European Union', 'Austria', 'Belgium', 'Belgium/Luxembourg',
       'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia',
       'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland',
       'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta',
       'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia',
       'Slovenia', 'Spain', 'Sweden', 'Japan', 'Norway',
       'Russian Federation', 'Switzerland', 'Tunisia', 'United Kingdom',
       'United States of America'], dtype=object)

## Exporting to csv files

In [22]:
# saving files to csv

# df_total_production = df_list[0] # total production by country
# df_domestic_consumption = df_list[1] # consuption by export
# df_export = df_list[2]
# df_import = df_list[3]
# df_consumption_in_import = df_list[4]

def saving_files_to_csv():
    """
    Saving files straight to csv files from our dataset.list
    
    """
    
    df_total_production.to_csv(r'C:\Users\JASNYP\Desktop\Python Projects\GlobalCoffeeTrade\cleaned_dataset\coffee_production.csv', index = False )
    df_domestic_consumption.to_csv(r'C:\Users\JASNYP\Desktop\Python Projects\GlobalCoffeeTrade\cleaned_dataset\coffe_consumption_by_exporting.csv', index = False ) 
    df_export.to_csv(r'C:\Users\JASNYP\Desktop\Python Projects\GlobalCoffeeTrade\cleaned_dataset\coffee_export_by_country.csv', index = False )
    df_import.to_csv(r'C:\Users\JASNYP\Desktop\Python Projects\GlobalCoffeeTrade\cleaned_dataset\coffee_import_by_country.csv', index = False )
    df_consumption_in_import.to_csv(r'C:\Users\JASNYP\Desktop\Python Projects\GlobalCoffeeTrade\cleaned_dataset\coffee_consumption_by_importing.csv', index = False )

saving_files_to_csv()


## Export to excel files

In [23]:

def saving_files_to_excel():

    """
    Saving files to excel file - every dataset in separate sheet 
    """


    writer = pd.ExcelWriter(r'C:\Users\JASNYP\Desktop\Python Projects\GlobalCoffeeTrade\cleaned_dataset\coffee_trade.xlsx', engine = 'xlsxwriter')
    df_total_production.to_excel(writer, sheet_name = 'Production by country', index = False)
    df_domestic_consumption.to_excel(writer, sheet_name = 'Domestic consumption', index = False)
    df_export.to_excel(writer, sheet_name = 'Export by country', index = False)
    df_import.to_excel(writer, sheet_name = 'Import by country', index = False)
    df_consumption_in_import.to_excel(writer, sheet_name = 'Consumption by import country', index = False)

    writer.close()

saving_files_to_excel()

In [55]:
df_import.Country.unique()

array(['European Union', 'Austria', 'Belgium', 'Belgium/Luxembourg',
       'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia',
       'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland',
       'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta',
       'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia',
       'Slovenia', 'Spain', 'Sweden', 'Japan', 'Norway',
       'Russian Federation', 'Switzerland', 'Tunisia', 'United Kingdom',
       'United States of America'], dtype=object)

In [63]:
country_list = ['European Union', 'Germany', 'United States of America', 'Poland']

In [67]:
temp_index = df_import.set_index('Country')
temp_index

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total_import
Country,,,,,,,,,,,,,,,,,,,,,
European Union,42882.776087,2.468632e+09,2.671800e+09,2.715268e+09,2.724128e+09,2.605463e+09,2.784504e+09,2.871915e+09,2.932127e+09,2.984879e+09,...,3.963163e+09,4.088211e+09,4.081840e+09,4.313592e+09,4.319702e+09,4.584148e+09,4.453282e+09,4.689073e+09,4.803423e+09,1.024337e+11
Austria,1879.717004,1.234963e+08,1.323850e+08,1.101893e+08,8.503284e+07,7.388616e+07,7.262796e+07,7.765220e+07,7.760125e+07,8.998662e+07,...,8.714957e+07,9.351656e+07,9.329562e+07,9.150486e+07,9.080342e+07,8.969631e+07,8.759469e+07,8.576441e+07,8.685082e+07,2.653189e+09
Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.801803e+08,...,3.497065e+08,3.400939e+08,3.301286e+08,3.127050e+08,3.320109e+08,3.631302e+08,3.406427e+08,3.445352e+08,3.719338e+08,6.240480e+09
Belgium/Luxembourg,2014.954017,1.047735e+08,1.096711e+08,1.237956e+08,1.357484e+08,1.440899e+08,1.515494e+08,1.526628e+08,2.147807e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.137071e+09
Bulgaria,268.182998,1.201320e+07,1.093386e+07,2.384658e+07,2.776008e+07,3.092508e+07,1.632960e+07,1.764924e+07,2.052732e+07,2.201514e+07,...,2.889061e+07,3.358950e+07,3.656920e+07,3.724254e+07,4.090540e+07,4.674960e+07,4.241295e+07,4.423464e+07,4.709380e+07,8.146848e+08
Croatia,NaN,NaN,1.009140e+07,9.766440e+06,1.160700e+07,1.920792e+07,1.931016e+07,2.311782e+07,2.034306e+07,2.112498e+07,...,2.345244e+07,2.306942e+07,2.477096e+07,2.518041e+07,2.623209e+07,2.872000e+07,2.634446e+07,2.673082e+07,2.798505e+07,6.221167e+08
Cyprus,49.853999,3.235560e+06,2.452560e+06,4.644720e+06,3.110160e+06,2.626140e+06,3.090120e+06,2.480460e+06,2.986740e+06,3.327516e+06,...,4.924725e+06,5.032648e+06,5.625971e+06,5.663286e+06,5.358293e+06,5.589198e+06,5.996032e+06,7.990773e+06,7.993492e+06,1.273487e+08
Czechia,659.433000,3.151644e+07,2.848386e+07,3.085662e+07,3.322782e+07,3.319182e+07,3.151788e+07,3.361350e+07,4.589034e+07,4.366061e+07,...,5.714200e+07,6.759737e+07,7.103294e+07,1.093614e+08,1.240703e+08,1.067611e+08,9.041803e+07,1.141895e+08,9.653529e+07,1.776551e+09
Denmark,959.105004,5.896542e+07,6.369738e+07,5.963682e+07,6.199104e+07,5.262954e+07,6.035995e+07,5.694423e+07,6.082339e+07,6.562676e+07,...,5.453218e+07,5.478594e+07,5.527074e+07,5.120543e+07,5.044666e+07,5.434738e+07,5.113160e+07,5.207703e+07,5.447611e+07,1.684298e+09


In [71]:
temp_index.loc[country_list].Total_import

Country
European Union              1.024337e+11
Germany                     3.066112e+10
United States of America    4.124721e+10
Poland                      4.789462e+09
Name: Total_import, dtype: float64